# *Prediction of cooling degree days for metropolitan areas: prioritising corrective measures to deal with heat waves* (Cameroon Group)

<b> This notebook is attached to a report with the same title, as part of the course *Modern Data Analytics [G0Z39a]*. The goal of this report is to, given various datasets, gain a rapid insight into how different variables (territorial, socio-economic or environmental) can be used to predict a metropolitan area's cooling degree days (CDD). This parameter, coined by the OECD, refers to the amount of days per year in which threshold room temperature is exceeded, and to what extent. The indicator thus expresses an energy demand for the cooling of buildings. As data scientists, we'll be looking to what parameters are most contributing to that phenomena, and what regression models can be used to approximate the observed variations in CDD best. This notebook pertains the following structure: 

1. Import packages
2. Data preparation
    * Metropolitan data
    * National data
    * Data formatation
    * Data Cleaning
    * Outlier detection and removal
3. Data Analysis
   * Data Exploration
   * Clustering Analysis
   * Time Series Analysis
4. Model Calibration
   * Variable Selection
   * Modeling pipeline
5. Model Validation
    
    
</b>

# 1. Import packages

In [ ]:
#Packages
import numpy as np
import sys
import pandas as pd
import wbgapi as wb
import sklearn.preprocessing
import seaborn as sns
from pandas import DataFrame
from scipy.stats import shapiro
from sklearn.covariance import MinCovDet
import sys
import os
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
import scipy as scy
import pandas as pd
from time import time
#from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LassoCV
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso, PoissonRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
import seaborn as sb
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans
%matplotlib inline


#Own .py-scripts
%run Country_City_Correspondance.py
%run Modeling_Functions.py

# 2. Data Preparation
## 2.1 Import Data
<b> Note that a special webservice 'wbgapi' is used to import data from the WorldBank, to assist the formatting process </b>

In [ ]:
# Import OECD-data from CSV-files

#Metropolitan-based data
oecd_data1 = pd.read_csv('Data/CITIES_POPULATION.csv', sep="|", header=0)
oecd_data2 = pd.read_csv('Data/CITIES_AGE.csv', sep="|", header=0)
oecd_data3 = pd.read_csv('Data/CITIES_ECONOMY.csv', sep="|", header=0)
oecd_data4 = pd.read_csv('Data/CITIES_LABOUR.csv', sep="|", header=0)
oecd_data5 = pd.read_csv('Data/CITIES_TERRITORY.csv', sep="|", header=0)
oecd_data6 = pd.read_csv('Data/CITIES_ENVIRONMENT.csv', sep="|", header=0)

#Country-based data: this will later be translated into metropolitan data
country_codes = pd.read_excel('Data\COUNTRY_CODE.xls')
country_inlandwater = pd.read_csv('Data\COUNTRY_INLAND_WATER.csv')
country_evapotranspiration = pd.read_csv('Data\COUNTRY_EVAPOTRANSPIRATION.csv')
country_roaddensity = pd.read_csv('Data\COUNTRY_ROAD_DENSITY.csv')
country_area = wb.data.DataFrame('AG.LND.TOTL.K2',time=2018,labels=True)


## 2.2. Metropolitan data (OECD) 

In [ ]:
# Put all OECD-data into dataframe and append all dataframes
oecd_data_df = [oecd_data1, oecd_data2, oecd_data3, oecd_data4, oecd_data5, oecd_data6]
# Call concat method
oecd_df = pd.concat(oecd_data_df)

<b>The data is not structured in the way we want to process it (too many columns, multiple rows per sample,...). Above all, the data needs to be structured in a comprehensible format (rows=samples, columns=variables). Furthermore, a linear combination of some variables will be useful for later analysiss and merging with country-based data.

Priority is given to the primary process, such that the final structuring can be done on the bulk of the adapted data.

In [ ]:
oecd_df.head()

### 1.2.1 Variable creation as a linear combination of existing variabels

<b> A variable 'Construction ratio' needs to be created, in order to fetch the relative built environment

In [ ]:
## CONSTRUCTION RATIO

#pick the Territory dataset, calculate the construction densiy ratio
oecd_data5_1 = oecd_data5.loc[oecd_data5['Variables']=='Urbanised area (built-up area or land for urban use in km2)']
oecd_data5_2 = oecd_data5.loc[oecd_data5['Variables']=='Metropolitan area total land area']

#oecd_data5_1 has 1336 rows, while oecd_data5_2 has 668 rows (exactly two time of the first dataset), 
#we could try to combine the two datasets by the common column "METRO_ID",

oecd_data5_improved = pd.merge(oecd_data5_1,oecd_data5_2,how='inner',on='METRO_ID')

#and then calculate the constuction density ratio = "Urbanised area" / "Metropolitan area total land area"
oecd_data5_improved['construction_ratio'] = oecd_data5_improved['Value_x'] / oecd_data5_improved['Value_y']
oecd_data5_improved.head(3)

### 1.2.2 Rename columnames

In [ ]:
# create a dictionary where key = old name and value = new name
dict = {'METRO_ID': 'metroId',
        'Metropolitan areas': 'metropolitanAreas',
        'VAR': 'var',
       'Variables' : 'variables',
        'TIME' : 'time',
        'Year' : 'year',
        'Unit Code': 'unitCode',
        'Unit' : 'unit',
        'PowerCode Code' : 'powerCodeCode',
        'PowerCode': 'powerCode',
        'Reference Period Code' : 'referencePeriodCode',
        'Reference Period' : 'referencePeriod',
        'Value':'value',
        'Flag Codes' : 'flagCodes',
        'Flags': 'flags'
       }
  
# call rename () method
oecd_df.rename(columns=dict,
          inplace=True)

In [ ]:
# Checking the data
print(oecd_df.info())

### 1.2.3 Add longitude and latitude to join data

<b> In order to add the possibility of visualisation, longitude and lattitude is computed for every metropolitan area

Strings that represent metropolitan areas are often different, which makes it difficult to join data from different sources. For example, "Lafayette (IN)" and "Lafayette, IN" and "Lafayette". Therefore, longitude and latitude are added as a unique identifier to join data. 

This is performed by *LatLongGeneration.py*. Since the script carries out computationally intensive tasks, it is left out of this code and thus only the result is imported as a .csv-file. </b>


<b> Now, we add longitudes and latitudes to metropolitan areas </b>

In [ ]:
df_metropolitan = pd.read_csv('Data/CITIES_COORD.csv', header=0, sep="|", doublequote=True)
df_metropolitan

<b> Merge and create temporary dataset, without country-based statistics

In [ ]:
# merge the dataframes
df_oecdCoord = pd.merge(oecd_df, df_metropolitan, on="metropolitanAreas")

## 2.3 Country-based statistics (Worldbank, OECD)

<b> A first step in the process of adding national statistics as proxys for metropolitan statistics, is to built a metropolitan area-country correspondance matrix. The idea being that all national indicators can then be added to that dataframe, only then to be weighed correctly according to the relative surface of a metropolitan area in a country.

This of course assumes all variables (such as inland water, see later) to be homogeneously present over an entire country. Therefore this disaggregation process entails overestimation in some cases, underestimation in others. We consider it elligeble proxy's nonetheless.</b>

### 2.3.1 Metropolitan area - country correspondance and exception handling

<b> This process is partly automated by making use of the abbreviations utilized by OECD. Exception handling is however necessary, since some abbreviations for countries differ per dataset.

In [ ]:
#OECD set of population contains the best reference in terms of number of unique cities
cities_set = set(oecd_data1['METRO_ID'])

country_dict = country_codes[['CODE','Country']].to_dict('records')
country_dict

In [ ]:
#Run external .py-script which attaches country names to each metropolitan area
df_country_city = country_cities_correspondance(country_dict,cities_set)

### 2.3.2 Disaggregating national statistics

In [ ]:
#Prepare extra variables for merging with data

#Inland water
inland_water_15 = country_inlandwater[country_inlandwater['Year']==2015]
inland_water_15 = inland_water_15[inland_water_15['MEAS']=='PCNT']
inland_water_15 = inland_water_15[['Country','Value']]

#Road density
road_density = country_roaddensity[['Country','Amount']]

#Evapotranspiration
evapotrans = country_evapotranspiration[['Country','Value']]

#merge first two
road_and_water = pd.merge(inland_water_15,road_density,how="inner",on='Country')
road_water_evapo = pd.merge(road_and_water,evapotrans,how="inner",on='Country')

#merge with main dataframe
df_country_city_ext = df_country_city.reset_index().merge(road_water_evapo, how="left",on='Country').set_index('index')
df_country_city_ext.rename(columns = {'Value_x':'CNTRY_INLAND_WATER','Amount':'CNTRY_ROAD_DENS','Value_y':'EVAPOTRANS'}, inplace = True)
df_country_city_ext

In [ ]:
#Now we add relative territory of the city

cities_surface = oecd_data5[oecd_data5['VAR']=='SURF']
cities_surface = cities_surface[['METRO_ID','Value']]
cities_surface.rename(columns = {'METRO_ID':'index'}, inplace = True)
cities_surface

country_area = country_area[['Country','AG.LND.TOTL.K2']]

#merge both with the main dataframe
merged_data = df_country_city_ext.reset_index().merge(country_area, how="left",on='Country').set_index('index')
merged_data = merged_data.reset_index().merge(cities_surface, how="left",on='index').set_index('index')

merged_data.rename(columns = {'AG.LND.TOTL.K2':'CNTRY_TOT_AREA','Value':'CITY_TOT_SURF'}, inplace = True)
merged_data

#Calculate relative city surface
merged_data['CITY_REL_SURF'] = (merged_data['CITY_TOT_SURF'] / merged_data['CNTRY_TOT_AREA'])

In [ ]:
#Use relative city surface to weigh and disaggregate national statistics
merged_data['CITY_REL_ROADS'] = (merged_data['CNTRY_ROAD_DENS'].astype(float) * merged_data['CITY_REL_SURF'])
merged_data['CITY_REL_WATER'] = (merged_data['CNTRY_INLAND_WATER'] * merged_data['CITY_REL_SURF'])
merged_data['CITY_EVAPOTRANS'] = (merged_data['EVAPOTRANS'] * merged_data['CITY_REL_SURF'])

#Prepare for merge with city-based statistics
merged_data.rename(index = {'index':'metroId'}, inplace = True)
merged_data

## 2.4. Transpose dataset, merge datasets

<b> Now, we're going to transform the matrix such that on the rows we find countries, whereas on the columns we find variables.
In principle, such matrix could be built for every year. A first step is to delete all metadata related variables. Next, only the useful variables are kept. A final step is to eliminate the time dimension.

In [ ]:
#first get rid of all metadata-related columns
df_oecd_col_red = df_oecdCoord[['metroId','metropolitanAreas','latitude','longitude','var','variables','year','value']]

#first test some things on 1 year
df_oecd_col_red_14 = df_oecd_col_red[df_oecd_col_red['year']==2014]
#df_oecd_col_red_14.head()

#group by metropolitanAreas
df_total_14 = df_oecd_col_red_14[['metroId','metropolitanAreas','latitude','longitude','var','value']]

df_total_14 = df_total_14.pivot_table(index=['metroId','metropolitanAreas','latitude','longitude'], 
                    columns=['var',], 
                    values='value', 
                    aggfunc='mean')
df_total_14

<b> Once metadata is cleared, and the dataset is pivotted, it is time to perform a preliminary QUALITATIVE variable selection, since the OECD-data contains many variables already that are highly unlikely to contribute to the model given the scope of this research (e.g. vulnerability to pollution (PWM_EX_CORE) is taken in stead of specific vulnerability to specific pollution). Preference is furthermore given to aggregate and relative variables (e.g. population density, or share of elderly in stead of absolute numbers). </b>

In [ ]:
#Now we only select the useful variables: i.e. aggregate variables, relative variables, ...
df_selection_14 = df_total_14[['CDD','URB_AREA','URB_AREA_CORE',
                               'URB_AREA_HINTER','FRAGMENTATION',
                               'GDP_PC_REAL_PPP','PARTIC_RA_15_64','POP_DEN',
                              'POP_TOT_GI','PWM_EX_CORE','T_Y0_14_SH_NAT',
                               'T_Y15_64_SH_NAT','T_Y65_MAX_SH_NAT'
                              ]]
df_selection_14


<b> We now add some variables seperately, notably: the construction data from 1.2.2, and the tree share cover from the territory dataset (OECD). It is noted that the 'Tree share cover' data is from 2015, whereas the general dataset contains data from 2014. Although tree cover data is available from 2004, such that an interpolation would be possible, we opted not to do so, since interpolation or extrapolation entails callibrating a growth model (linear, logarithmic, exponential?). Since such computations would induce more errors.</b>

In [ ]:
#Next we add (1) construction ratio and (2) Tree cover manually as variables

#Construction_data
construction_ratio_14 = oecd_data5_improved[oecd_data5_improved['Year_x']==2014]
construction_ratio_14 = construction_ratio_14[['METRO_ID','construction_ratio']]

#Tree_cover
tree_cover = oecd_data6[oecd_data6['VAR']=='TREECOVER_SHARE_CORE']
tree_cover = tree_cover[tree_cover['Year']==2015]
tree_cover = tree_cover[['METRO_ID','Value']]

tree_and_construction = pd.merge(construction_ratio_14,tree_cover,how='inner',on='METRO_ID')
tree_and_construction = tree_and_construction.rename(columns = {'METRO_ID':'metroId','construction_ratio': 'CONSTR_RAT', 'Value': 'TREECOVER_SHARE_CORE'}, inplace = False)

#Merge with main dataset

df_extended_14 = df_selection_14.reset_index().merge(tree_and_construction, how='left',on='metroId')
df_extended_14


Next, the country-based statistics are to be added to the dataset

In [ ]:
#create a selection of only the variables to be merged
national_data = merged_data[['CITY_REL_ROADS','CITY_REL_WATER','CITY_EVAPOTRANS']]
national_data.reset_index(level=0, inplace=True)
national_data = national_data.rename(columns={'index':'metroId'})
national_data

In [ ]:
complete_data = df_extended_14.reset_index().merge(national_data, how='left',on='metroId')
complete_data

# 2.5. Data Cleaning

In [ ]:
complete_data.info()

In [ ]:
complete_data.describe()

In [ ]:
complete_data.isna().sum()

<b> Samples for which we don't have any data on the Cooling Degree Days, are to be dropped anyway since they will not be of any use as observations for the the dependent variable. </b>

In [ ]:
complete_data.dropna(subset=['CDD'],inplace=True)
# drop values that have missing data for cooling days
complete_data.head(10)

In [ ]:
# We further reduce the dataset extent by only allowing columns to exhibit at least 80% real values.
complete_data.dropna(thresh=0.8*len(complete_data),axis=1,inplace=True)

#replace missing values with mean of the specific column
complete_data_1 = complete_data.iloc[:,5:].where(pd.notna(complete_data), complete_data.mean(), axis="columns")


In [ ]:
complete_data_1.isna().sum()

In [ ]:
complete_data_1.insert(0, 'metroId', complete_data['metroId'])
complete_data_1.insert(1,'metropolitanAreas',complete_data['metropolitanAreas'])
complete_data_1.insert(2,'lat',complete_data['latitude'])
complete_data_1.insert(3,'long',complete_data['longitude'])

data_clean1 = complete_data_1
complete_data_1.head()

## 2.6 Outlier detection & removal

In [ ]:
complete_data_1.describe()

In [ ]:
shapiro(complete_data_1['CDD'])

<b> As the shapiro test yields a p-value smaller than 0.05, the null hypothesis that the variable CDD is normally distributed is rejected. Therefore, when detecting outliers, we should note that any method that asumes normal distribution (such as using  Z-scores) is to be disregarded.</b>

In [ ]:
sns.histplot(complete_data_1['CDD'])
#axes-level function for histograms

<b> The Minimum Covariance Determinant covariance estimator is to be applied on Gaussian-distributed data,
but could still be relevant on data drawn from a unimodal, symmetric distribution. 
It is not meant to be used with multi-modal data (the algorithm used to fit a MinCovDet object is likely to fail in such a case). One should consider projection pursuit methods to deal with multi-modal datasets. </b>


In [ ]:
sns.histplot(complete_data_1['CDD'])
#axes-level function for histograms
#unimodal

In [ ]:
dist_test_mincov = MinCovDet(random_state=0).fit(complete_data_1.loc[:,'CDD':'CITY_REL_WATER']).mahalanobis(complete_data_1.loc[:,'CDD':'CITY_REL_WATER'])

In [ ]:
from scipy.stats import chi2
crit_distance = chi2.ppf((1-0.01), df=complete_data_1.shape[1] - 1)
#p-value that is less than .001 is considered to be an outlier

In [ ]:
#idx = dist_test_mincov>crit_distance
crit_distance

In [ ]:
complete_data_mahalonobis = complete_data_1.copy()

In [ ]:
complete_data_mahalonobis['mahalanobis'] = dist_test_mincov

In [ ]:
complete_data_mahalonobis

In [ ]:
idx = dist_test_mincov>crit_distance
np.sum(idx==True)

<b> Above represents the number of ouliers removed based on Mahalanobis Distance </b>

In [ ]:
sns.histplot(complete_data_mahalonobis['CDD'])

In [ ]:
data_clean2 = complete_data_mahalonobis[complete_data_mahalonobis['mahalanobis'] < crit_distance]

# 3. Data Analysis

In [ ]:
data_clean1.head()

In [ ]:
data_clean2.head()

In [ ]:
df_main1 = data_clean1.loc[:,"CDD":"CITY_EVAPOTRANS"]
df_main2 = data_clean2.loc[:,"CDD":"CITY_EVAPOTRANS"]

In [ ]:
df_main1 = df_main1.where(pd.notna(df_main1), df_main1.mean(), axis="columns")
df_main2 = df_main2.where(pd.notna(df_main2), df_main2.mean(), axis="columns")

In [ ]:
df_main1.corr()

In [ ]:
fig = px.histogram(data_clean1, x="CDD")
fig.show()

In [ ]:
fig = px.histogram(data_clean2, x="CDD")
fig.show()

## 3.2. Time series analysis: Checking for Autocorrelation in Cooling Days

In [ ]:
oecd_df.head(3)

In [ ]:
#Get CDD series for New York
ny_data = oecd_df.loc[oecd_df["metropolitanAreas"] == 'New York (Greater)']

In [ ]:
cooling_days_ny = ny_data.loc[ ny_data["var"] == 'CDD',["year", "value"]]
cooling_days_ts = cooling_days_ny.set_index('year')

In [ ]:
def tsplot(y, lags=None, figsize=(10, 8), style='bmh'):
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    with plt.style.context(style):    
        fig = plt.figure(figsize=figsize)
        layout = (3, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))
        qq_ax = plt.subplot2grid(layout, (2, 0))
        pp_ax = plt.subplot2grid(layout, (2, 1))
        
        y.plot(ax=ts_ax)
        ts_ax.set_title('Time Series Analysis Plots')
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.5)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.5)
        sm.qqplot(y, line='s', ax=qq_ax)
        qq_ax.set_title('QQ Plot')        
        scs.probplot(y, sparams=(y.mean(), y.std()), plot=pp_ax)

        plt.tight_layout()
    return 

In [ ]:
tsplot(np.diff(cooling_days_ts.value),lags=8)

<b> We can safely conclude that there is no autocorrelation in the Target variable CDD (Cooling days). However, we do note that the data is not suitable for time series analysis, due to the low resolution. </b>

## 3.3. Clustering
<b> Cluster analysis is a statistical technique aimed to uncover groups (clusters) of observations that are homogeneous and separated from other groups. For this project, it might be useful to group a large number of cities by the values of their population and the number of yearly cooling days. The group of primary interest is the one that covers cities with the largest number of population and cooling days (the most vulnerable to heat waves). </b>

In [ ]:
# Drop the rows with missing values
dfc=df_main1.dropna(axis=0)

In [ ]:
#Choose the variables for the analysis
scaler = StandardScaler()
scaler.fit(df_main1[['CONSTR_RAT','CDD']])

df = scaler.transform(df_main1[['CONSTR_RAT','CDD']])
X = df

#df = df_main1[['CONSTR_RAT','CDD']]
#df = df_main[['POP_DEN','CDD']]
#df = df_main[['GDP_PC_REAL_PPP','CDD']]
#df = df_main.loc[:,"CDD":"CITY_EVAPOTRANS"]
#dfc = df_main[['POP_TOT_GI','CDD']]
#df = df_main[['T_Y0_14_SH_NAT','CDD']]
#df = df_main[['T_Y15_64_SH_NAT','CDD']]

<b> K-means clustering method is applied. The desired number of clusters (k) needs to be specified in advance. The K-means algorithm then assigns each observation to exactly one of the k clusters. The algorithm is run multiple times from different random initial configurations aimed to find the best separated clusters. The K-means algorithm lacks flexibility in cluster shape and probabilistic cluster assignment. </b>

<b> For clustering, the data needs to be standardized given that the variables in the data set have different scales and variances. The dataset has already been standardized by using StandardScaler(). Clusters with centeroids are visualized then on the scatterplot. </b>

In [ ]:
# Initialize Kmeans for 2 clusters and fit it to the data
cluster_with_scaling = KMeans(n_clusters=2, n_init=20,init='random',random_state=0)
cluster_with_scaling.fit(X)

In [ ]:
y=cluster_with_scaling.predict(X)
fig,ax = plt.subplots(figsize=(10,10))
idx_1 = y==1
idx_0 = y==0
color_1='blue'
color_0 ='red'
ax.scatter(X[idx_1,0],X[idx_1,1],s=40,color=color_1,label=None)
ax.scatter(X[idx_0,0],X[idx_0,1],s=40,color=color_0,label=None)

ax.scatter(cluster_with_scaling.cluster_centers_[:,0],cluster_with_scaling.cluster_centers_[:,1],s=120,
           color='k',marker='s',
           label='Centroids of the Clusters')

ax.set_xlabel('Construction Rate')
ax.set_ylabel('Cooling Days');
ax.legend();

<b> As seen from the scatter plots, there are cities that could be susceptible to heat waves and damages associated with heatwaves more than cities in other groups. For example, cities with population larger than 135 thousands and cooling days largerthan 730 days (calculated by using the values of the mean and standard deviation of the dataset before scalling applied) </b>

# 4. Model Calibration

## 4.1. Variable selection

In [ ]:
## 4.1. Variable selection# create new arrays for variable importance scaled data with outliers
y_1 = df_main1.loc[:,"CDD"]
X_1 = df_main1.loc[:,"URB_AREA":"CITY_EVAPOTRANS"]
X_1train, X_1test, y_1train, y_1test = train_test_split(X_1, y_1,test_size=0.2,random_state=0)

In [ ]:
np.array(X_1train.columns)

In [ ]:
#Initialise a new scaling object
sc=StandardScaler() 

# Set up the scaler just on the training set
X_train = sc.fit_transform(X_1train)

In [ ]:
lasso = LassoCV(cv=10, random_state=0,max_iter=10000).fit(X_train, y_1train)
importance = np.abs(lasso.coef_)
feature_names = np.array(X_1train.columns)
plt.figure(figsize=(15,8))
plt.bar(height=importance, x=feature_names)
plt.title("Feature importances via coefficients")
plt.xticks(rotation=90)
plt.show()

In [ ]:
# create new arrays for variable importance scaled data with outliers removed
y_2 = df_main2.loc[:,"CDD"]
X_2 = df_main2.loc[:,"URB_AREA":"CITY_EVAPOTRANS"]
X_2train, X_2test, y_2train, y_2test = train_test_split(X_2, y_2,test_size=0.2,random_state=0)

In [ ]:
#Initialise a new scaling object
sc1=StandardScaler() 

# Set up the scaler just on the training set
X_train = sc1.fit_transform(X_2train)

In [ ]:
lasso_2 = LassoCV(cv=10, random_state=0,max_iter=10000).fit(X_train, y_2train)
importance_2 = np.abs(lasso_2.coef_)
feature_names = np.array(X_2train.columns)
plt.figure(figsize=(15,8))
plt.bar(height=importance_2, x=feature_names)
plt.title("Feature importances via coefficients")
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Array for calibration with outliers
y = df_main1.loc[:,"CDD"]
X = df_main1.loc[:,"URB_AREA":"CITY_EVAPOTRANS"]
rng = np.random.RandomState(0)
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=rng)

# Array for calibration with outliers removed
y_1 = df_main2.loc[:,"CDD"]
X_1 = df_main2.loc[:,"URB_AREA":"CITY_EVAPOTRANS"]
rng = np.random.RandomState(0)
X_1train, X_1test, y_1train, y_1test = train_test_split(X_1, y_1,test_size=0.2,random_state=rng)

## 4.2. Modeling pipeline

In [ ]:
# data with outliers
pre_process = ColumnTransformer(remainder='passthrough',
                                transformers=[('drop_columns', 'drop', ['T_Y0_14_SH_NAT',
                                                                        'URB_AREA',
                                                                        'T_Y15_64_SH_NAT',
                                                                        'CONSTR_RAT',
                                                                        'TREECOVER_SHARE_CORE',
                                                                        'CITY_REL_WATER'
                                                                       ])])

In [ ]:
#data without outliers
pre_process_2 = ColumnTransformer(remainder='passthrough',
                                transformers=[('drop_columns', 'drop', ['T_Y0_14_SH_NAT',
                                                                        'T_Y15_64_SH_NAT',
                                                                        'URB_AREA'
                                                                       ])])

In [ ]:
 cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

In [ ]:
 cv_1 = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

<b> both initiators below: *Scale_predictor_variables()* and *Scoring()* belong to the Modeling_Functions.py helper script </b>

In [ ]:
scale_predictor_variables = Scale_predictor_variables(X_train,y_train,pre_process,cv)

In [ ]:
scale_predictor_variables.Plot_cross_validation_results()

In [ ]:
scale_predictor_variables = Scale_predictor_variables(X_1train,y_1train,pre_process_2,cv_1)

In [ ]:
scale_predictor_variables.Plot_cross_validation_results()

In [ ]:
Scoring1 = Scoring(pre_process,X_train,y_train)

In [ ]:
Scoring1.Score()

In [ ]:
Scoring2 = Scoring(pre_process_2,X_1train,y_1train)

In [ ]:
Scoring2.Score()

# 5. Model Validation

<b> The *Model_select()* is also to be found within Modeling_Functions.py </b>

In [ ]:
model_select = Model_select(X_train,y_train,X_test, y_test)
#with outliers

In [ ]:
model_select.model_selection()

In [ ]:
model_select.models_score
#R-squared

In [ ]:
model_select_1 = Model_select(X_1train,y_1train,X_1test, y_1test)
#without outliers

In [ ]:
model_select_1.model_selection()

In [ ]:
model_select_1.models_score
#R-squared

In [ ]:
selected_model_1 = GradientBoostingRegressor(learning_rate=0.04, max_depth=4, random_state=0,
                          subsample=0.5)
selected_model_1.fit(X_1train,y_1train)
pd.DataFrame({'Variable':X_1train.columns,
              'Importance':selected_model_1.feature_importances_}).sort_values('Importance', ascending=False)

In [ ]:
selected_model_2 = RandomForestRegressor(max_depth=20, max_features=2, n_estimators=80,
                      random_state=0)
selected_model_2.fit(X_1train,y_1train)
pd.DataFrame({'Variable':X_1train.columns,
              'Importance':selected_model_2.feature_importances_}).sort_values('Importance', ascending=False)